In [ ]:
import utils
import sim
import plot_helpers as ph
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import plotly.graph_objects as go
import plotly.express as px
import os


In [ ]:
def split_range(range_endpoints, step_mult):
    range_endpoints = [np.linspace(range_endpoints[i], range_endpoints[i+1], num = step_mult, endpoint=False) for i in range(len(range_endpoints)-1)]
    return np.array(range_endpoints).flatten()


import sim
import importlib
importlib.reload(sim)

RUN_TOGETHER_ONLY = False
N_RUNS = 20

settings = {
    'figure-2A': {
        'energy_split_type': 'equal',
        'delta': 0
    },
    'figure-2B': {
        'energy_split_type': 'equal',
        'delta': 0.01
    },
    'figure-2C': {
        'energy_split_type': 'random::uniform',
        'delta': 0
    },
    'figure-2B': {
        'energy_split_type': 'random::uniform',
        'delta': 0.01
    },
}

figure = 'figure-2A'
delta = settings[figure]['delta']
energy_split_type = settings[figure]['energy_split_type']

if energy_split_type == 'equal':
    energy_subfolder = 'equal-energy-split'
elif energy_split_type == 'random::uniform':
    energy_subfolder = 'random-energy-split'
else:
    raise ValueError('energy split strategy not recognized')

subfolder = f'delta={delta:<.3f}'

bm_step = 0.01
rf_step = 0.01
bm_0 = 0.2
rf_0 = 0.6
bm_range = [bm_0 + i * bm_step for i in range(1, 19)]
rf_range = [rf_0 + i * rf_step for i in range(1, 19)]

print('-'*60)
print(f'bm_range: {bm_range}')
print(f'rf_range: {rf_range}')

grid_folder_name = (f'grid -- bm_0={bm_0:.2f} vs [{bm_range[0]:.2f}, {bm_range[-1]:.2f}, steps={len(bm_range)}]'
                    f' rf_0={rf_0:.2f} vs [{rf_range[0]:.2f}, {rf_range[-1]:.2f}, steps={len(rf_range)}]'
                    f' runs={N_RUNS}'
                    # f' st [{st_range[0]}, {st_range[-1]} steps={steps}]'
                )
print(f'grid_folder_name: {grid_folder_name}')



In [ ]:
job_prefix = os.path.join(
    'jobs',
    # 'random-energy-split',
    energy_subfolder,
    'one-vs-all',
    grid_folder_name, 
    subfolder,
    sim.get_timestamp())

base_cell_params = {
        "type": "low-fidelity",
        "initial-energy": 1,
        "energy-threshold-for-split": 5,
        "single-period-energy-accu": 1,
        "cell-dies-in-round-prob": delta,
        "kill-both-on-bad-replication": True,
        "base-metabolism-energy": bm_0,
        "replication-fidelity": rf_0,
        'energy-distribution-on-split': energy_split_type
    }
settings = {
    "simulation": {
        "number-of-runs": N_RUNS,
        "number-of-rounds": 3*10**3,
        "initial-cell-count": 30,
        "gather-stats-every": 50,
        "print-progress": 0.0,
        "stop-when-one-type-wins": True
    },
    "env": {
        "type": "flow",
        "regen-rate": 30,
        "regen-probability": 1.0
    },
}

print(f'len(bm_range): {len(bm_range)}')
print(f'len(rf_range): {len(rf_range)}')

pbar = tqdm(total=len(bm_range) * len(rf_range), leave=True)
sim.run_one_vs_grid(
    job_prefix=job_prefix,
    settings=settings,
    base_cell_params=base_cell_params,
    bm_range=bm_range,
    rf_range=rf_range,
    # st_range=None,
    pbar=pbar,
    run_together_only=RUN_TOGETHER_ONLY, 
    create_figure_for_each_run=True
)


In [ ]:
rows = utils.grid_simulations_rows(job_prefix)
df = pd.DataFrame(rows)
print(f'len(df): {len(df)}')
display(df)
df = df.loc[(df['low-fidelity'] != 0) | (df['high-fidelity'] != 0), :]
df.to_csv(os.path.join(job_prefix, 'results.csv'), index=False)
